# Modules to be Installed
The following modules are required for this project:

- `os`
- `sys`
- `gmsh`
- `numpy` (`np`)
- `pandas` (`pd`)
- `pyvista` (`pv`)
- `matplotlib.pyplot` (`plt`)
- `math`
- `meshio`
- `gmshparser`
- `importlib`
-  `math`
- `shapely.geometry`
```markdown
To install these modules, use the following pip command:

```bash
pip install os sys gmsh numpy pandas pyvista matplotlib math meshio gmshparser importlib

```

In [106]:
# appel fonction mesh
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 18 14:57:23 2023
import functions mesh
"""
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
# Ajouter le chemin du dossier src au PYTHONPATH
import importlib
sys.path.append('/home/ariviere/Programmes/ginette')  # Ajouter le dossier parent de src
from src.src_gmsh import mesh_generator
from src.src_python import Direct_model
from src.src_python import Init_folders
importlib.reload(mesh_generator)
from src.src_gmsh.mesh_generator import plot_gmsh_mesh,process_distance_altitude,generate_mesh_8_region,remove_elements_above_curve_all_entities,voisin_mesh,coord_to_row_column,compute_am_bm_from_row_col
from src.src_python.Init_folders import *
from src.src_python.Direct_model import *
#print functions of Init_folders


importlib.reload(Init_folders)
# give me all names of the functions in Init_folders
#print(Init_folders.__dict__)
#print([name for name in dir(Init_folders) if callable(getattr(Init_folders, name)) and not name.startswith("__")])

compile_ginette()

In [10]:
# Configuration du répertoire de travail
# Utiliser os.getcwd() pour obtenir le répertoire de travail actuel
script_dir = os.getcwd()  # Répertoire du notebook
mesh_dir = os.path.join(script_dir, "meshdata")  # Chemin absolu vers "meshdata"
sim_dir= script_dir
file_path = "/home/ariviere/Documents/Bassin_Orgeval/Donnee_Orgeval_Mines/raw_data/DESC_data/DATA_STATION/"  # Chemin vers le fichier CSV

# Ajouter le chemin du dossier src au PYTHONPATH
sys.path.insert(0, "/home/ariviere/Programmes/ginette/src")
# Paramètres de du maillage
## name de la station
Station = "IntD"
# Paramètres sens de la station par rapport à la direction de la rivière
#       |   ↑    |
#  RG   |   ↑    |  RD
#       |        |
#  Rive_D=0 station à l'envers x_piezo_RD=0m dans l'exemple du schema
Rive_D=0
plot=True
meshplot=True
altitude_min = 103
dz_pt100=0.15
dz_pt100first=0.1



In [11]:
# reading the data georef
distance_altitude_table,pt100_coord= process_distance_altitude(file_path, Station, mesh_dir, altitude_min, Rive_D,dz_pt100=dz_pt100,dz_pt100first=dz_pt100first)    

print("coordonnées des capteurs de températures")
print(pt100_coord)
# definition des zones zones
#zone 1 
#zone 2
permeabilite_2 = 2.5662199999999999E-08 
rect_2 = np.array([[0, 103.], [16, 103.],[16., 103.83],[0, 103.83]])
# zone 3
permeabilite_3 = 9.9999999999999995E-07
rect_3= np.array([[0, 103.83], [16, 103.83],[16, 105.02],[0, 105.02]])
# zone 4
permeabilite_4=0.0001200000000000000
poly_4=np.array([[10.7,104.7],[9.67076,104.40417],[9.48341,104.37706],[9.3,104.35],
                 [8.82518,104.25082],[8.7,104.22],[8.35,104.22],[7.3,104.22],
                 [6.8,104.22],[6.4,104.28],[6.35819,104.28925],
                 [5.91353,104.42771],[5.8,104.5],[5.65,104.6],
                 [5.94139,103.83],
               [6.8832,103.83],[7.40838,103.83],[8.06661,103.83], 
                 [9.3,103.83],[9.48341,103.83]
                ])
# zone 5        
permeabilite_5=4.9899999999999997E-08 
poly_5=np.array([[10.7,104.7],[9.67076,104.40417],[9.48341,104.37706],[9.3,104.35],
                 [8.82518,104.25082],[8.7,104.22],[8.35,104.22],[7.3,104.22],
                 [6.8,104.22],[6.4,104.28],[6.35819,104.28925],
                 [5.91353,104.42771],[5.8,104.5],[5.65,104.6],[5.5,104.8],
                 [5.8,104.7],[6.1,104.7],[6.4,104.6],[6.8,104.6],
                 [7.3,104.5],[7.7,104.5],[8.3,104.5],[8.7,104.6],[9.3,104.5],[10.92239,104.8]]) 
# 
# 
#    
# plot des données
if plot:
    import matplotlib.pyplot as plt

    # Configure the plot
    plt.figure(figsize=(12, 8))
    plt.plot(
        distance_altitude_table["Distance (m)"],
        distance_altitude_table["Altitude (Z)"],
        marker='o',
        # add name distance_altitude_table["Instrument"] if not NA
        linestyle='-',
        color='blue',
        label='Altitude Profile'
    )

    # Ajouter les noms des instruments au-dessus des points si ce n'est pas "NA"
    for i, instrument in enumerate(distance_altitude_table["Instrument"]):
        if instrument != "NA" and not pd.isna(instrument):
            plt.text(
                distance_altitude_table["Distance (m)"][i],
                distance_altitude_table["Altitude (Z)"][i] + 0.1,  # Décalage vertical pour éviter de chevaucher le point
                instrument,
                fontsize=10,
                color='red',
                ha='center'
            )
    # Tracer les zones rectangulaires xmin ymin and xmax ymax

    plt.fill(rect_2[:, 0], rect_2[:, 1], color='green', alpha=0.3, label='Zone 2')
    plt.fill(rect_3[:, 0], rect_3[:, 1], color='blue', alpha=0.3, label='Zone 3')
    plt.fill(poly_4[:, 0], poly_4[:, 1], color='orange', alpha=0.3, label='Zone 4')
    plt.fill(poly_5[:, 0], poly_5[:, 1], color='purple', alpha=0.3, label='Zone 5')

# Tracer les capteurs de température PT100
# Tracer les capteurs de température PT100
    if not pt100_coord.empty:
        pt100_label_added = False  # Variable pour suivre si le label "PT100" a été ajouté
        for i in range(len(pt100_coord)):
            plt.scatter(
                pt100_coord["Distance (m)"][i],
                pt100_coord["Altitude (Z)"][i],
                color='red',
                label="PT100" if not pt100_label_added else None  # Ajouter le label uniquement une fois
            )
            pt100_label_added = True  # Marquer que le label a été ajouté
    # Add titles and labels    # Initialize the neighbors as -99 (no neighbor)
    # The neighbors are stored in the ivois DataFrame
    plt.title("Distance vs Altitude", fontsize=16, fontweight='bold')
    plt.xlabel("Distance (m)", fontsize=14)
    plt.ylabel("Altitude (Z)", fontsize=14)

    # Add grid and legend
    plt.grid(True, linestyle='--', alpha=0.7)
 # Ajouter la légende en dehors de la figure, à droite
    plt.legend(fontsize=12, loc='upper left', bbox_to_anchor=(1, 0.5))

    # Show the plot
    plt.tight_layout()
    plt.show()
#     





In [15]:
import math
## altitude min du maillage
x_hoboRG=6.8
x_hoboRD=8.7
z_hoboRG=104.5
z_hoboRD=104.5
x_RG=5.25
x_RD=10.25
z_riv=104.575
v_bot=103.925
dx_precis=0.10
dx_grossier=0.25
dz_grossier=0.25
dz_precis=0.05
dx_hobo=0.05
 # Calcul des subdivisions en x pour les regions centre 2 et 5
length_x_centre = x_RD-x_RG
num_div_x_centre = max(2, math.ceil(length_x_centre / dx_precis))
max_x=distance_altitude_table['Distance (m)'].max()
min_x=distance_altitude_table['Distance (m)'].min()
max_z=distance_altitude_table['Altitude (Z)'].max()
# Calcul des subdivisions en x pour les regions de gauche 1, 4 et 7
length_x_gauche = x_RG - min_x
num_div_x_gauche = max(2, math.ceil(length_x_gauche / dx_grossier))

        # Calcul des subdivisions en x pour les regions centre 2 et 5
length_x_centre = x_RD- x_RG
num_div_x_centre = max(2, math.ceil(length_x_centre / dx_precis))
# Calcul des subdivisions en x pour les regions droite 3, 6 et 8
length_x_droite = max_x - x_RD
num_div_x_droite = max(2, math.ceil(length_x_droite / dx_grossier))

        # Calcul des subdivisions en z pour les regions haute 7 et 8
num_div_z_haut = max(2, math.ceil((max_z - z_riv) / dz_grossier))
        # Ensure the number of subdivisions is an integer by adjusting the step size
if (max_z - z_riv) % dz_grossier != 0.1:
    num_div_z_haut = max(2, round((max_z - z_riv) / dz_grossier))
        
        # Calcul des subdivisions en z pour les regions centre 4, 5 et 6
num_div_z_centre = max(2, math.ceil((z_riv - v_bot) / dz_precis))

        # Calcul des subdivisions en z pour les regions bas 1,2 et 3
num_div_z_bas = max(2, math.ceil((v_bot - altitude_min)/dz_grossier))

print(f"Subdivisions calculées :\n"
            f"  - Gauche : {num_div_x_gauche} subdivisions\n"
            f" - dx gauche : {length_x_gauche / num_div_x_gauche:.2f} m\n"
            f"  - Centre : {num_div_x_centre} subdivisions\n"
            f" - dx centre : {length_x_centre / num_div_x_centre:.2f} m\n"
            f"  - Droite : {num_div_x_droite} subdivisions\n"
            f" - dx droite : {length_x_droite / num_div_x_droite:.2f} m\n"
            f"  - Haut : {num_div_z_haut} subdivisions\n"
            f" - dz haut : {(max_z - z_riv) / num_div_z_haut:.2f} m\n"
            f"  - Centre : {num_div_z_centre} subdivisions\n"
            f" - dz centre : {(z_riv - v_bot) / num_div_z_centre:.2f} m\n"
            f"  - Bas : {num_div_z_bas} subdivisions\n"
            f" - dz bas : {(v_bot - altitude_min) / num_div_z_bas:.2f} m\n")

In [16]:
if meshplot:
    # Appel de la fonction
    output_mesh_path = os.path.join(mesh_dir, "rectangular_mesh.msh")
    generate_mesh_8_region(distance_altitude_table, output_mesh_path,v_bot=v_bot,x_RG =x_RG, x_RD = x_RD,z_riv=z_riv, 
                           dx_grossier=dx_grossier, dx_precis = dx_precis,x_hobo_1 =x_hoboRG, z_hobo_1=z_hoboRG,x_hobo_2 = x_hoboRD,z_hobo_2=z_hoboRD, dx_hobo=dx_hobo, 
                           dz_grossier=dz_grossier, dz_precis=dz_precis,dz_hobo1=dz_pt100first,dz_hobo=dz_pt100,mesh_dimension=2)

    output_mesh_path = os.path.join(mesh_dir, "rectangular_mesh.msh")
    remove_elements_above_curve_all_entities(distance_altitude_table, output_mesh_path, ajout='_remove_all_entities', retirer='')
    plot_gmsh_mesh(output_mesh_path.replace(".msh", "_remove_all_entities.msh")) # Modification du nom car on applique 2 fois la fct remove elements 



# creation fichier coordonnées

In [17]:

from src.src_gmsh import mesh_generator
importlib.reload(mesh_generator)

from src.src_gmsh.mesh_generator import readGmsh
fName = os.path.join(mesh_dir, "rectangular_mesh_remove_all_entities.msh")
# rename to pt100_coord with x and z
pt100_coord.rename(columns={"Distance (m)": "x", "Altitude (Z)": "z"}, inplace=True)

centres,nb_mesh = readGmsh(fName, precision=3)
# write the coordinates in a file witout the header
centres.to_csv(os.path.join(sim_dir, "E_coordonnee.dat"), sep=' ', header=False, index=False)


# write index with the first =1 in the first column of the centres dataframe
centres['index'] = np.arange(1, len(centres) + 1)

print(centres.head())


### Figure des centres de mailles

In [27]:
x = centres['x']
z = centres['z']
fig, ax = plt.subplots(figsize = (16,9))

ax.scatter(x,z,marker="+") # Trace le centre de chaque mailles
plt.show()

### Index = nmaille ginette to save

In [28]:

# find index of the centre near the pt100_coord and add the number of centre in pt100_coord
for i in range(len(pt100_coord)):
    # find the index of the centre nearest to pt100_coord
    index = ((centres['x'] - pt100_coord["x"][i])**2 + (centres['z'] - pt100_coord["z"][i])**2).idxmin()
    # add the index to the pt100_coord
    pt100_coord.loc[i, 'index'] = centres.loc[index, 'index']
    # add the distance to the pt100_coord
    pt100_coord.loc[i, 'distance'] = np.sqrt((centres['x'][index] - pt100_coord["x"][i])**2 + (centres['z'][index] - pt100_coord["z"][i])**2)
    pt100_coord.loc[i, 'xmaille'] = centres.loc[index, 'x']
    pt100_coord.loc[i, 'zmaille'] = centres.loc[index, 'z']


pt100_coord['n_maille'] = pt100_coord['index'].astype(int)


print(pt100_coord.head(8))

In [37]:

# Filter to zoom in on the desired region
centres_zoom = centres[
    (centres['x'] > 6.6) & (centres['x'] < 7) &
    (centres['z'] > 104.) & (centres['z'] < 104.6)
]

# Plot the points
plt.figure(figsize=(12, 8))
plt.scatter(centres_zoom['x'], centres_zoom['z'], c=centres_zoom['index'], cmap='viridis', s=30)

# Add text labels for each point
for _, row in centres_zoom.iterrows():
    plt.text(row['x'], row['z'], str(row['z']), fontsize=20, color='black', ha='center', va='center')

# Add axis labels and a colorbar for clarity
plt.xlabel('x (meters)')
plt.ylabel('z (meters)')
plt.title('Zoom on Mesh Centres with Index Numbers')
plt.colorbar(label='Mesh Number')
plt.show()

# Tableau de voisinage

In [38]:
importlib.reload(mesh_generator)
ivois = voisin_mesh(script_dir)

## test voisinage

In [40]:


# Filter to zoom in on the desired region
centres_zoom = centres[
    (centres['x'] > 6.6) & (centres['x'] < 7) &
    (centres['z'] > 104.) & (centres['z'] < 104.6)
]

# Plot the points
plt.figure(figsize=(12, 8))
plt.scatter(centres_zoom['x'], centres_zoom['z'], c=centres_zoom['index'], cmap='viridis', s=30)

# Add text labels for each point
for _, row in centres_zoom.iterrows():
    plt.text(row['x'], row['z'], str(row['index']), fontsize=20, color='black', ha='center', va='center')

# Add axis labels and a colorbar for clarity
plt.xlabel('x (meters)')
plt.ylabel('z (meters)')
plt.title('Zoom on Mesh Centres with Index Numbers')
plt.colorbar(label='Mesh Number')
plt.show()


# creation dimension

In [41]:

# Call the function coord_to_row_column
nb_col,nb_row=coord_to_row_column(script_dir)

In [42]:
importlib.reload(mesh_generator)
compute_am_bm_from_row_col(script_dir)

# creation des E_zone.dat
* E_zone.dat: numero zone de chaque maille 1 ligne = 1 interger
    '''
    Fonction qui permet de remplir le fichier E_zone.dat utile pour ginette. On a besoin pour cela du maillage (fichier E_coordonee.dat) et d'un dataframe comprenant
    les polygones rangés selon leur zone respectivent. La fonction test si le centre de chaque maille se trouve ou non dans un polygone. Une fois la zone de la maille
    trouvé c'est ajouté à une liste qui en fin de fonction sera écrit dans le fichier E_zone.dat

    Entrée
    path_E_coor (str) : Chemin (relatif ou absolu) amenant au fichier E_coordonee.dat
    path_E_zone (str) : Chemin (relatif ou absolu) amenant au fichier E_zone.dat
    polygons_by_zone (DataFrame) : Df comprenant les polygones qui définissent les différentes zones du maillage. Un exemple de création de polygons_by_zone est donné dans la suite du code
    default_zone (int) : zone (par default = 1) que l'on va attribuer au point contenu dans aucun polygone (Evite de définir tout les polygone dans polygons_by_zone)
    '''

# Récuppérer les numéros des centres des PT100 pour les indiquer dans E_parametres.dat

In [43]:
print(nb_mesh)
# E_zone.dat with the number 1 for each mesh 
# need to be adapted with the nnumber of zone
# dataframe with integer 1 for each mesh
zone = pd.DataFrame(np.ones((nb_mesh, 1), dtype=int), columns=['zone'])
# print E_zone.dat 
zone.to_csv(os.path.join(mesh_dir, "E_zone.dat"), sep=' ', header=False, index=False)

In [44]:
importlib.reload(Direct_model)
setup_ginette_perm_2D(pt100_coord,nb_mesh,nb_col,nb_row)
initial_conditions_perm_2D()

# creation des conditions aux limites
* Piezo rive gauche rive droite pour la charge :       
      open (unit=222, file='E_BordRD.dat', iostat=iidRD)
      open (unit=223, file='E_BordRG.dat', iostat=iidRG)
      open (unit=2242, file='E_bottomZH.dat', iostat=iidZH)
      open (unit=224, file='E_Id_river.dat', iostat=iidrv)
      open (unit=2244, file='E_Id_river_max.dat', iostat=iidrvtest)
      open (unit=225, file='E_TempSol.dat', iostat=iTsol)
      open (unit=226, file='E_PluieR.dat', iostat=i226)
      open (unit=227, file='E_chargeT_RD.dat', iostat=iCRD)
      open (unit=228, file='E_chargeT_RG.dat')
!     open(unit=229,file='E_tempT_RD.dat')
!     open(unit=230,file='E_tempT_RG.dat')
!     open(unit=231,file='E_tempT_Riv.dat')
      open (unit=232, file='E_chargeT_Riv.dat')
Riviere hauteur d'eau dans la rivière
flux nul en bas flux nul en haut hormis la rivière

Faire tourner un régime permanant pour générer les conditions initiales

# Premiere simulation calculer le temps de simulation

Grid search avec seulement la permeabilté de la zone 4 et 5 au début.
Ploter pour les 8 capteurs de températures les rmse

Ensuite grid search sur k, porosité et conductivité thermique.

Tester sur une autre periode contrasté.